<a href="https://colab.research.google.com/github/VesalAhsani/Driver-behavior-detection/blob/main/DMS_YOLOv8s_Focal_224_RGB%26GRAY_NoAugmentation_TruePretrained_062025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1️⃣ Install dependencies
# Colab comes with torch+cuda preinstalled, but we upgrade ultralytics.
!pip install --upgrade ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.5 MB/s eta 0:00:00


In [2]:
# @title 2️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# @title 3️⃣ Paths & Unzip Dataset
import zipfile, os, glob
from pathlib import Path

# ── CONFIG ──────────────────────────────────────
ZIP_PATH    = "/content/drive/MyDrive/Split_rgb&gray_NotAugmented_224.zip"
EXTRACT_DIR = Path("/content/dataset224")
DATA_DIR    = EXTRACT_DIR / "Split224"   # <-- must contain train/ & val/
# ────────────────────────────────────────────────

# 3.1 Unzip if needed
if not DATA_DIR.exists():
    print("📦 Unzipping dataset…")
    EXTRACT_DIR.mkdir(exist_ok=True)
    with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
        zf.extractall(EXTRACT_DIR)
    print("✅ Unzipped to", EXTRACT_DIR)

# 3.2 Print counts per class
print("\n🧾 Dataset composition:")
for split in ("train", "val"):
    print(f"\n– {split.upper()}:")
    for cls in sorted((DATA_DIR/split).iterdir()):
        if cls.is_dir():
            imgs = list(cls.glob("*.*"))
            print(f"  {cls.name:<20s} {len(imgs):5d}")

📦 Unzipping dataset…
✅ Unzipped to /content/dataset224

🧾 Dataset composition:

– TRAIN:
  Control_Panel         8341
  Drinking              7823
  Eating                2114
  Look_Down             3761
  Makeup                6219
  Normal                8167
  Phone_Call_(Left)     7125
  Phone_Call_(Right)    7459
  Reaching_Behind       4634
  Sleep                 4039
  Smoking_Left          4126
  Smoking_Mouth         4169
  Smoking_Right         4060
  Talk_to_passengers    5137
  Text_(Left)           5106
  Text_(Right)          5130
  Yawning               3206

– VAL:
  Control_Panel          927
  Drinking               870
  Eating                 235
  Look_Down              418
  Makeup                 691
  Normal                 908
  Phone_Call_(Left)      792
  Phone_Call_(Right)     829
  Reaching_Behind        515
  Sleep                  449
  Smoking_Left           459
  Smoking_Mouth          464
  Smoking_Right          452
  Talk_to_passengers     571
  Te

In [4]:
# @title 4️⃣ Compute α (class imbalance) for focal loss

import torch

# count images per class
train_root = DATA_DIR / "train"
class_counts = {
    cls.name: len(list((train_root/cls.name).glob("*.*")))
    for cls in sorted(train_root.iterdir()) if cls.is_dir()
}

# compute inverse-frequency weights, cap at 2.2
max_n     = max(class_counts.values())
raw_alpha = torch.tensor([max_n / class_counts[c] for c in class_counts],
                         dtype=torch.float32)
alpha     = torch.minimum(raw_alpha, torch.tensor(2.2)).cuda()

print("\n🎯 α weights (per class, clipped):")
for name, w in zip(class_counts.keys(), alpha):
    print(f"  {name:<20s} {w:.3f}")


🎯 α weights (per class, clipped):
  Control_Panel        1.000
  Drinking             1.066
  Eating               2.200
  Look_Down            2.200
  Makeup               1.341
  Normal               1.021
  Phone_Call_(Left)    1.171
  Phone_Call_(Right)   1.118
  Reaching_Behind      1.800
  Sleep                2.065
  Smoking_Left         2.022
  Smoking_Mouth        2.001
  Smoking_Right        2.054
  Talk_to_passengers   1.624
  Text_(Left)          1.634
  Text_(Right)         1.626
  Yawning              2.200


In [5]:
# @title 5️⃣ Define FocalLoss & Train YOLOv8-s-cls

import torch
import torch.nn as nn
import shutil
from pathlib import Path
from ultralytics import YOLO

# ─── Focal Loss ───────────────────────────────
class FocalLoss(nn.Module):
    def __init__(self, gamma=1.5, alpha=None, reduction='mean'):
        super().__init__()
        self.gamma, self.alpha, self.reduction = gamma, alpha, reduction

    def forward(self, inputs, targets):
        logp = torch.nn.functional.log_softmax(inputs, dim=-1)
        p    = torch.exp(logp)
        t    = targets.long()
        pt   = p.gather(-1, t.unsqueeze(-1)).squeeze(-1)
        focal = (1 - pt) ** self.gamma
        loss = -focal * logp.gather(-1, t.unsqueeze(-1)).squeeze(-1)
        if self.alpha is not None:
            loss = loss * self.alpha[t]
        return loss.mean() if self.reduction=='mean' else loss.sum()

# ─── Load model & override loss ─────────────────
print("🚀 Loading YOLOv8s-cls pretrained weights…")
model = YOLO("yolov8s-cls.pt")
model.model.loss_fn = FocalLoss(gamma=1.5, alpha=alpha)
print("🔍 Active loss_fn:", model.model.loss_fn)

# ─── Train ────────────────────────────────────
print("\n📚 Starting training…")
model.train(
    data=str(DATA_DIR),     # expects train/ & val/ inside
    epochs=50,
    batch=32,
    imgsz=224,
    lr0=5e-4,
    optimizer="Adam",
    project="/content/drive/MyDrive/yolov8s_cls_224_results",
    name="run_focal224",
    pretrained=True,
    # no horizontal flips unless you want mirror invariance
    fliplr=0,
    flipud=0,
)


# ─── Validate & copy best.pt ──────────────────
print("\n📊 Running final validation…")
metrics = model.val()

src = os.path.join(model.trainer.save_dir, "weights", "best.pt")
dst = os.path.join(RESULTS_DIR, run_name, "weights", "best.pt")
os.makedirs(os.path.dirname(dst), exist_ok=True)
os.system(f"cp {src} {dst}")

print("✅ Best weights saved to:", dst)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Loading YOLOv8s-cls pretrained weights…


100%|██████████| 12.3M/12.3M [00:00<00:00, 398MB/s]


🔍 Active loss_fn: FocalLoss()

📚 Starting training…
Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset224/Split224, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0, flipud=0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run_focal224, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, pa

100%|██████████| 5.35M/5.35M [00:00<00:00, 78.8MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 631.1±243.6 MB/s, size: 14.1 KB)


...

               classes   top1_acc   top5_acc: 100%|██████████| 158/158 [00:03<00:00, 43.65it/s]


                   all      0.992          1
Speed: 0.1ms preprocess, 0.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8s_cls_224_results/run_focal224

📊 Running final validation…
Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,096,977 parameters, 0 gradients, 12.5 GFLOPs
train: /content/dataset224/Split224/train... found 90616 images in 17 classes ✅ 
val: /content/dataset224/Split224/val... found 10076 images in 17 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 565.9±235.0 MB/s, size: 14.1 KB)


val: Scanning /content/dataset224/Split224/val... 10076 images, 0 corrupt: 100%|██████████| 10076/10076 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 315/315 [00:04<00:00, 78.15it/s]


                   all      0.992      0.999
Speed: 0.1ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8s_cls_224_results/run_focal2242


AttributeError: 'dict' object has no attribute 'project'

In [ ]:
import os
import time
from google.colab import runtime

# Wait for training to complete before disconnecting
print("Training completed! Releasing GPU resources...")
time.sleep(20)  # Give some time for final processing

# Automatically disconnect the Colab session
runtime.unassign()
os._exit(0)